In [28]:
import os
os.getcwd()


'd:\\Electricity-Consumption-Prediction'

In [29]:
#os.chdir(path='Electricity-Consumption-Prediction')

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    base_model_path : Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_loss_function : str
    params_optimizer : str

In [32]:
from electrycityconsumption.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from electrycityconsumption.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training_model
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = Path(self.config.data_ingestion.local_data_path)
       

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data= training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_loss_function=params.LOSS_FUNCTION,
            params_optimizer=params.OPTIMIZER

        )

        return training_config



In [35]:
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd

class Training:
    def __init__(self, config : TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )

    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)

    def load_training_data(self,training_data_path):
        try:
            # Load your training data from the provided CSV file
            data = pd.read_csv(training_data_path)

            # Assuming your data has columns for features and a target variable 'Usage_kWh'
            # Replace with the actual column names from your data
            X = data.drop(columns=['Usage_kWh'])  # Exclude the target variable
            y = data['Usage_kWh']  # Target variable

          

            return X,y
        except Exception as e:
            print(f"Error loading training data: {str(e)}")
            return None, None

    
    
    
    def train(self):
        self.get_base_model()
        #Load your training data
        X, y = self.load_training_data(self.config.training_data)
        #print(y)
        # Split your data into training and validation sets
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
        X_valid = tf.convert_to_tensor(X_valid, dtype=tf.float32)
        y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
        y_valid = tf.convert_to_tensor(y_valid, dtype=tf.float32)
        X_train = tf.reshape(X_train, [-1, 32, 14])
        y_train = tf.reshape(y_train, [-1, 32, 1])
        X_valid = tf.reshape(X_valid, [-1, 32, 14])
        y_valid = tf.reshape(y_valid, [-1, 32, 1])
        # Train your model
        self.model.fit(X_train, y_train, epochs=self.config.params_epochs, validation_data=(X_valid, y_valid))

        # Save your trained model
        self.save_model(self.config.trained_model_path, self.model)

In [36]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train()
except Exception as e:
    raise e

2023-10-26 01:36:59,481: INFO : common :yaml file: electrycityconsumption\config\config.yaml loaded successfully
2023-10-26 01:36:59,517: INFO : common :yaml file: electrycityconsumption\params.yaml loaded successfully
2023-10-26 01:36:59,532: INFO : common :created directory at: artifacts


AttributeError: EagerTensor object has no attribute 'reshape'. 
        If you are looking for numpy-related methods, please run the following:
        tf.experimental.numpy.experimental_enable_numpy_behavior()
      

In [ ]:
os.getcwd()

'd:\\'